# Calculating the distance between the Customer's city and the Seller's city

In [1]:
from pyspark.sql import SparkSession, functions as F
import math

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
orders_items_df = spark.read \
                 .option('escape', '\"') \
                 .option('quote', '\"') \
                 .csv('./dataset/olist_order_items_dataset.csv', header=True, multiLine=True, inferSchema=True)

orders_df = spark.read \
                 .option('escape', '\"') \
                 .option('quote', '\"') \
                 .csv('./dataset/olist_orders_dataset.csv', header=True, multiLine=True, inferSchema=True)

customers_df = spark.read \
                  .option('escape', '\"') \
                  .option('quote', '\"') \
                  .csv('./dataset/olist_customers_dataset.csv', header=True, multiLine=True, inferSchema=True)

sellers_df = spark.read \
                  .option('escape', '\"') \
                  .option('quote', '\"') \
                  .csv('./dataset/olist_sellers_dataset.csv', header=True, multiLine=True, inferSchema=True)

geo_df = spark.read \
                  .option('escape', '\"') \
                  .option('quote', '\"') \
                  .csv('./dataset/olist_geolocation_dataset.csv', header=True, multiLine=True, inferSchema=True)

# Grouping data

In [11]:
data_df = orders_df.filter(F.col('order_status') == 'delivered').join(customers_df, 'customer_id')

data_df = orders_items_df.join(data_df, 'order_id') \
                         .join(sellers_df, 'seller_id') \
                         .select('customer_state', 'customer_city', 'customer_zip_code_prefix', 'seller_state', 'seller_city', 'seller_zip_code_prefix', 'freight_value')

data_df = data_df.join(geo_df, data_df.customer_zip_code_prefix == geo_df.geolocation_zip_code_prefix) \
                 .select(F.col('geolocation_lat').alias('customer_lat'), F.col('geolocation_lng').alias('customer_lng'), 'seller_state', 'seller_city', 'seller_zip_code_prefix', 'freight_value') \
                 .join(geo_df, data_df.seller_zip_code_prefix == geo_df.geolocation_zip_code_prefix) \
                 .select('customer_lat', 'customer_lng', F.col('geolocation_lat').alias('seller_lat'), F.col('geolocation_lng').alias('seller_lng'),'freight_value')

In [5]:
data_df.show()

+-------------------+------------------+-------------------+-------------------+-------------+
|       customer_lat|      customer_lng|         seller_lat|         seller_lng|freight_value|
+-------------------+------------------+-------------------+-------------------+-------------+
|-23.494190060505503|-47.40954169025909| -23.54195269441404| -46.65931762155928|        14.43|
|-23.494190060505503|-47.40954169025909| -23.54443774132954| -46.65181342394259|        14.43|
|-23.494190060505503|-47.40954169025909|-23.542944348058064| -46.65793705274388|        14.43|
|-23.494190060505503|-47.40954169025909| -23.54339898029315| -46.65151833472787|        14.43|
|-23.494190060505503|-47.40954169025909| -23.54323868858565| -46.65731599111321|        14.43|
|-23.494190060505503|-47.40954169025909|-23.544121128772638| -46.65531989785911|        14.43|
|-23.494190060505503|-47.40954169025909| -23.54052443880199| -46.65050381848346|        14.43|
|-23.494190060505503|-47.40954169025909| -23.54143

# Calculating distance

In [ ]:
def d(c_lat, c_lng, s_lat, s_lng):
    radius = 6371 # km

    dlat = math.radians(s_lat-c_lat)
    dlon = math.radians(s_lng-c_lng)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(c_lat)) \
        * math.cos(math.radians(s_lat)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c;

    return d

distance = F.udf(d)

data_df = data_df.withColumn('distance', distance('customer_lat', 'customer_lng', 'seller_lat', 'seller_lng'))

data_df.show()

In [ ]:
data_df.groupBy().agg(F.max(F.col('distance'))).show()